# Generating configs for width 64 model

In [9]:
fixed = {
    "m_L_ec": 6,
    "sector": None,
    "ec_loss": "haughty_focal",
    "lw_edge": 1,
    "m_alpha": 0.5,
    "adam_eps": 1e-08,
    "optimizer": "adam",
    "adam_beta1": 0.9,
    "adam_beta2": 0.999,
    "batch_size": 5,
    "ec_pt_thld": 0.9,
    "edge_indim": 4,
    "node_indim": 7,
    "focal_alpha": 0.3966639332867394,
    "focal_gamma": 3.9912747796867887,
    "adam_amsgrad": False,
    "n_graphs_val": 320,
    "val_data_dir": "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_9",
    "n_graphs_train": 247776,
    "train_data_dir": [
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_1",
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_2",
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_3",
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_4",
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_5",
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_6",
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_7",
        "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_8"
    ],
    "m_residual_type": "skip1",
    "adam_weight_decay": 0,
    "m_use_node_embedding": True,
    "m_use_intermediate_edge_embeddings": True,
}

In [10]:
from itertools import product

In [11]:
def get_scheduler_config(scheduler, start_lr, end_lr, t_max):
    if scheduler == "cosineannealinglr":
        return {
            "scheduler": scheduler,
            "cosineannealinglr_T_max": t_max,
            "lr": start_lr,
            "cosineannealinglr_eta_min": end_lr,
        }
    elif scheduler == "linearlr":
        return {
            "scheduler": scheduler,
            "linearlr_total_iters": t_max,
            "lr": start_lr,
            "linearlr_start_factor": 1,
            "linearlr_end_factor": end_lr / start_lr,
        }
    else:
        raise ValueError(f"Unknown scheduler {scheduler}")

In [12]:
from pathlib import Path
import json
import random

outdir = Path("/Users/fuchur/Documents/22/git_sync/gnn_tracking_hpo/config/")
outdir.mkdir(exist_ok=True, parents=True)

In [13]:
dims = [64]
start_lrs = [1e-3, 8e-4]
end_lrs = [1e-5, 5e-5]
beta1s = [0.9, 0.99]
schedulers = ["cosineannealinglr"]
t_maxs = [50]

In [14]:
variants = list(product(dims, start_lrs, end_lrs, schedulers, t_maxs, beta1s))
random.seed(42)
random.shuffle(variants)

configs = []
for dim, start_lr, end_lr, scheduler, t_max, beta1 in variants:
    config = fixed.copy()
    config.update(get_scheduler_config(scheduler, start_lr, end_lr, t_max))
    config["m_interaction_edge_dim"] = dim
    config["m_interaction_node_dim"] = dim
    config["m_hidden_dim"] = dim
    config["adam_beta1"] = beta1
    configs.append(config)
print(f"Generated {len(configs)} configs")

Generated 8 configs


In [15]:
for i, config in enumerate(configs, start=65):
    with open(outdir / f"config_{i}.json", "w") as f:
        json.dump(config, f, indent=4)